In [ ]:
first_question_article = [
     {'article_id': '00000194-25d6-dcc4-a1d7-3df6703c0000',
      'question': 'אני רוצה לקרוא ביקורת על הסרט החדש של לוקה גואדנינו, זה שמבוסס על ספר של בורוז על הומו שחי במקסיקו בשנות ה-50 ומחפש גברים צעירים.'},
      {'article_id': '00000194-2618-dd68-a3be-e6fc06680000',
       'question': 'אני מחפש ביקורת על הסרט החדש עם ניקול קידמן, זה שהיא משחקת בו מנכ"לית חברת רובוטיקה שיש לה רומן עם מתמחה צעיר. הסרט הוקרן בפסטיבל ונציה, איך הוא?.'
       },
       {'article_id': '00000194-2681-ddb6-afdd-77e781220000',
        'question': 'אני מחפש ביקורת על הסרט החדש של עמוס גיתאי, זה שעוסק בשאלה למה יש מלחמות ומשתמש במכתבים של איינשטיין ופרויד.'
        },
        {'article_id': '00000194-2b97-d9c2-a79e-2bd7330c0000',
            'question': 'אפשר בבקשה ביקורת על הסרט החדש "נוספרטו" של רוברט אגרס? אני רוצה לדעת אם הוא נאמן למקורות של סרטי הערפדים הקלאסיים ואיך השחקנים הראשיים, במיוחד לילי-רוז דפ, משחקים?'
        },
        {'article_id': '00000194-319b-d555-abbc-b1dfdf250000',
            'question': '"אני מחפש ביקורת על סרט חדש שמוקרן בפסטיבל חיפה, על אישה אלכוהוליסטית בשם רונה שחוזרת לאי הולדתה בסקוטלנד כדי להתמודד עם ההתמכרות שלה. מישהי כתבה על זה ביקורת?'
        },
        {
            'article_id': '00000194-31db-ddaf-adb7-7bfbaf200000',
            'question': 'אני מחפש סרט איראני חדש שמתרחש בטהרן ועוסק במשפחה שאבא שלה שופט ורואים בו גם קטעים מההפגנות שם. ראיתי שהקרינו אותו בפסטיבל חיפה'
        },
        {'article_id': '00000194-34df-d39d-a196-b7ff86ce0000',
            'question': '"אני מחפש סדרת אנימציה חדשה בנטפליקס, משהו בסגנון באפי קוטלת הערפדים אבל עם מיתולוגיה סינית. שמעתי שיש סדרה על נערה סינית-אמריקאית שנלחמת בשדים, מישהו מכיר?',
        },
        {
            'article_id':"00000194-356b-de88-a3dc-75ef26460000",
            'question':"אוקיי, שמעתי על סרט חדש של שבי גביזון עם ריצ'רד גיר על אבא שמגלה שהיה לו בן שנהרג. איך קוראים לסרט ומה הוא מספר עליו?"
        },
        {
        'article_id':'00000194-4be5-dff1-a7bc-dbf57a210000',
        'question':'אני מחפש סרט על שני בני דודים יהודים אמריקאים שנוסעים לפולין בעקבות השואה. אחד מהם משחק בסדרה "יורשים". מישהו כתב וביים את הסרט והוא גם שחקן בסרט "הרשת החברתית".',
        },
        {
        'article_id':'00000194-6339-dabc-afff-6b7fb6e90000',
        'question':'אוקיי, שמעתי שיש סדרה חדשה בנטפליקס, מין מערבון כזה אבל ממש קשוח, עם הרבה אלימות וטבח. זה על איזה אישה שמנסה לשרוד עם הבן שלה בתקופה של מלחמת יוטה והטבח בהר מדוז. מישהו יודע על מה אני מדבר?',
        },
        {
        'article_id':'00000194-6e60-d9d3-a597-eeecd15d0000',
        'question':'"אני מחפש מידע על הסרט החדש של ארז תדמור, זה שמתרחש בשנות ה-50 ליד הכנרת ועוסק בפרטיזנים לשעבר שעובדים במפעל סודה. איך קוראים לסרט ומה העלילה שלו?',
        },
]

first_question_genre = [
    "אחי, תמליץ לי על איזה סרט אקשן טוב בנטפליקס? משהו חדש יחסית.",
    "מחפשת סדרה קלילה ומצחיקה, קומדיה כזאת, לא משנה לי באיזה שירות סטרימינג.",
    "מה כדאי לראות עכשיו ביס פלוס? בא לי איזה משהו מתח טוב.",
    "יש לך המלצה לסרט דרמה איכותי? עדיף משהו שזמין בהוט.",
    "אני והחבר'ה רוצים לראות סרט אימה הערב. יש משהו ממש מפחיד שאתה מכיר?",
    "תגיד, יש איזה סדרת מדע בדיוני שווה בפרטנר TV או בסלקום TV?",
    "בא לי לראות סרט ישראלי טוב, יש משהו מומלץ לאחרונה?",
    "מחפש סדרה לבינג', משהו שאפשר לטחון בסופ\"ש. ז'אנר פחות קריטי, העיקר שתהיה ממכרת.",
    "יש סרטי דוקו מעניינים בדיסני פלוס או באפל טי וי?",
    "סתם תן המלצה לסרט טוב שראית לאחרונה, לא משנה לי הז'אנר או הפלטפורמה."

]

genre_questions_as_dicts = [
    {'article_id': '', 'question': q} for q in first_question_genre
]

# איחוד שתי הרשימות לרשימה אחת
combined_questions = first_question_article + genre_questions_as_dicts

second_question = 'איפה אני יכול לראות את הסרט/סדרה הזאת?'
thierd_question = 'איזה יצירה המבקר העדיף יותר?'



In [5]:
from src.llm_api_client import LLMClient
from config.load_config import load_config

prompts = load_config("config/prompts.yaml")
sys_instruct = prompts.get("system_instructions")
config = load_config("config/config.yaml")
llm_config = config.get("llm", {})
api_key = llm_config.get("GOOGLE_API_KEY")
model_name = llm_config.get("llm_model_name")

result = []

import numpy as np
counter = np.random.randint(0, 9999999999)

llm_client = LLMClient(model_name=model_name, api_key=api_key, sys_instruct=sys_instruct, config=config)
for q in combined_questions:
    print('\n')
    print("="*20)
    article_id = q.get('article_id')
    question = q.get('question')
    print(f"Article ID: {article_id}")
    print(f"Question: {question}")
    full_response = ""
    async for chunk in llm_client.streaming_message(question,f'{counter}'):
        full_response += chunk
        print(chunk, end='', flush=True)
    counter += 1

    second_answer = ""
    async for chunk in llm_client.streaming_message(second_question,f'{counter}'):
        second_answer += chunk
        print(chunk, end='', flush=True)


    thierd_answer = ""
    async for chunk in llm_client.streaming_message(thierd_question,f'{counter}'):
        thierd_answer += chunk
        print(chunk, end='', flush=True)

    result.append({
        'article_id': article_id,
        'question': question,
        'answer': full_response,
        'second_answer': second_answer,
        'thierd_answer': thierd_answer,
    })
    

/Users/roeishlezinger/Desktop/movie-recommendation-chatbot/src/tools/search/utillity/qdrant.py:9: UserWarning: Qdrant client version 1.13.3 is incompatible with server version 1.8.4. Major versions should match and minor version difference must not exceed 1. Set check_version=False to skip version check.
  self.client_qdrant = QdrantClient(config)
{"message": "Successfully connected to Redis."}
{"message": "No history found for user 5796282182 in Redis."}
{"message": "Successfully created new chat session."}




Article ID: 00000194-25d6-dcc4-a1d7-3df6703c0000
Question: אני רוצה לקרוא ביקורת על הסרט החדש של לוקה גואדנינו, זה שמבוסס על ספר של בורוז על הומו שחי במקסיקו בשנות ה-50 ומחפש גברים צעירים.


{"message": "Processing function call: find_movie_tv_info_or_review with args: {'query': '\u05d0\u05e0\u05d9 \u05e8\u05d5\u05e6\u05d4 \u05dc\u05e7\u05e8\u05d5\u05d0 \u05d1\u05d9\u05e7\u05d5\u05e8\u05ea \u05e2\u05dc \u05d4\u05e1\u05e8\u05d8 \u05d4\u05d7\u05d3\u05e9 \u05e9\u05dc \u05dc\u05d5\u05e7\u05d4 \u05d2\u05d5\u05d0\u05d3\u05e0\u05d9\u05e0\u05d5, \u05d6\u05d4 \u05e9\u05de\u05d1\u05d5\u05e1\u05e1 \u05e2\u05dc \u05e1\u05e4\u05e8 \u05e9\u05dc \u05d1\u05d5\u05e8\u05d5\u05d6 \u05e2\u05dc \u05d4\u05d5\u05de\u05d5 \u05e9\u05d7\u05d9 \u05d1\u05de\u05e7\u05e1\u05d9\u05e7\u05d5 \u05d1\u05e9\u05e0\u05d5\u05ea \u05d4-50 \u05d5\u05de\u05d7\u05e4\u05e9 \u05d2\u05d1\u05e8\u05d9\u05dd \u05e6\u05e2\u05d9\u05e8\u05d9\u05dd'}"}
{"message": "Query is likely Hebrew, returning original query."}
{"message": "Retrieving relevant documents for query: '\u05d0\u05e0\u05d9 \u05e8\u05d5\u05e6\u05d4 \u05dc\u05e7\u05e8\u05d5\u05d0 \u05d1\u05d9\u05e7\u05d5\u05e8\u05ea \u05e2\u05dc \u05d4\u05e1\u05e8\u05d8 \u05d4\

[{"article_id": "00000194-25d6-dcc4-a1d7-3df6703c0000", "article_name": "\"קוויר\": מרתק לראות את דניאל קרייג מפגין שילוב של חולשה וגבריות יהירה", "writer_name": ["פבלו אוטין"]}, {"article_id": "0000017f-e3db-df7c-a5ff-e3fbc9000000", "article_name": "אז איך זה נראה כשהוליווד המיינסטרימית עוסקת סוף סוף במתבגרים הומואים?", "writer_name": ["אורי קליין"]}]הנה ביקורת על הסרט "קוויר" של לוקה גואדנינו, המבוסס על ספרו של בורוז. הביקורת מאת פבלו אוטין מתארת את הסרט כמעין דיאלוג טלפתי, מיזוג יצירתי או מעשה אהבה בין גואדנינו לסופר הנערץ. דניאל קרייג מגלם את ויליאם לי, הומו אמריקאי המחפש גברים צעירים במקסיקו של שנות ה-50. הסרט מתואר כחוויה תודעתית ומשונה, המשלבת דימויים בורוזיאניים.

האם תרצה שאחפש ביקורות נוספות על סרטים של לוקה גואדנינו?

{"message": "No history found for user 5796282183 in Redis."}
{"message": "Successfully created new chat session."}


...

[{"article_id": "0000017f-db31-db5a-a57f-db7b992e0000", "article_name": "זאת הקומדיה הטובה של השנה - קורעת, מבריקה ומוזרה להפליא", "writer_name": ["אורון שמיר"]}, {"article_id": "0000017f-dbe1-d856-a37f-ffe160e50000", "article_name": "לראשונה זה ארבעה חודשים חזרתי לקולנוע. לא לסרט הזה פיללתי", "writer_name": ["אורי קליין"]}]אורון שמיר ממליץ על "בארב וסטאר יוצאות לויסטה דל מאר / עושות את פלורידה" (Barb and Star Go To Vista Del Mar). הוא מתאר אותה כקומדיה הטובה של השנה - קורעת, מבריקה ומוזרה להפליא, ובעלת פוטנציאל להפוך לסרט פולחן.

האם תרצה שאמצא ביקורות נוספות על הסרט הזה, או שאציע לך סרטים אחרים שאולי תאהב?


{"message": "No history found for user 5796282203 in Redis."}
{"message": "Successfully created new chat session."}
{"message": "Processing function call: find_movie_tv_info_or_review with args: {'query': '\u05d4\u05de\u05dc\u05e6\u05d4 \u05e2\u05dc \u05f4\u05d4\u05d9\u05e6\u05d5\u05e8\u05d9\u05dd \u05d4\u05d0\u05dc\u05d4\u05f4'}"}
{"message": "Query is likely Hebrew, returning original query."}
{"message": "Retrieving relevant documents for query: '\u05d4\u05de\u05dc\u05e6\u05d4 \u05e2\u05dc \u05f4\u05d4\u05d9\u05e6\u05d5\u05e8\u05d9\u05dd \u05d4\u05d0\u05dc\u05d4\u05f4'"}
{"message": "Tool 'find_movie_tv_info_or_review' executed successfully for query: '\u05d4\u05de\u05dc\u05e6\u05d4 \u05e2\u05dc \u05f4\u05d4\u05d9\u05e6\u05d5\u05e8\u05d9\u05dd \u05d4\u05d0\u05dc\u05d4\u05f4'. Results obtained."}
{"message": "Generated 1 function response parts."}


[{"article_id": "0000018a-b6e4-d3a1-a1eb-b7efb5c60000", "article_name": "\"הענקים של אי הפסחא\": דובר קוסאשווילי מגיע לגבהים חדשים", "writer_name": ["פבלו אוטין"]}, {"article_id": "0000017f-f590-d044-adff-f7f943eb0000", "article_name": "\"אלים זוטרים\": קובייה הונגרית קולנועית שחומקת מפתרון", "writer_name": ["נתנאל שלומוביץ"]}]המאמר "הענקים של אי הפסחא": דובר קוסאשווילי מגיע לגבהים חדשים מתאר את סרטו החדש של דובר קוסאשווילי, "הענקים של אי הפסחא", כסרט המשלב טון נוסטלגי ועדין עם פראות סגנונית אופיינית לקוסאשווילי. מי שיצליחו להיסחף בשינויים בין הוולגרי והקריקטורי לבין העדין והריאליסטי, יגלו סרט מבוים להפליא וכובש רגשית. הסרט מבוסס על אלמנטים אוטוביוגרפיים כדי לספר את סיפורה של משפחת עולים מגאורגיה, שגרה בשכונה באור יהודה אי שם בתחילת שנות ה-80.

המאמר השני, ""אלים זוטרים": קובייה הונגרית קולנועית שחומקת מפתרון" מתאר סרט דיסטופי עם נרטיב שמדלג קדימה ואחורה בלי להסגיר מה קדם למה, או מי חולם את מי. הסרט גדוש במסרים עמומים הנתונים לפרשנויות מרובות, ומבקר את תרבות הצריכה כפתרון גרוטסקי לבעיו

{"message": "Loaded history for user 5796282203 from Redis."}
{"message": "Successfully created new chat session."}


מתוך שני המאמרים שסיפקתי, המבקר פבלו אוטין העדיף יותר את "הענקים של אי הפסחא". הוא מתאר אותו כסרט מבוים להפליא וכובש רגשית, ומשבח את השילוב של טון נוסטלגי ועדין עם פראות סגנונית. לעומת זאת, נתנאל שלומוביץ' תיאר את "אלים זוטרים" כסרט מרתק לצפייה, אך עשוי לתסכל, ורחוק מלהיות סרט מושלם.

האם תרצה שאחפש השוואה מפורטת יותר בין שני הסרטים?

In [7]:
import pandas as pd
pd.DataFrame(result).to_excel('first_question_article.xlsx', index=False)